In [4]:
# | include: false
# | default_exp scilint

In [5]:
# | export

import ast
import os
import re
import shutil
import sys
from collections import Counter
from pathlib import Path
from typing import Iterable
import warnings

import nbformat
import numpy as np
import pandas as pd
from execnb.nbio import read_nb
from fastcore.script import call_parse
from nbdev.config import add_init, get_config
from nbdev.doclinks import _build_modidx, nbglob
from nbdev.export import nb_export
from nbdev.quarto import nbdev_docs, nbdev_readme
from nbqa.__main__ import _get_configs, _main
from nbqa.cmdline import CLIArgs
from nbqa.find_root import find_project_root

In [6]:
%load_ext autoreload
%autoreload 2

# Read-in Test Data

In [7]:
nbdev_path = Path(Path(".").resolve(), "example_nbs", "nbdev.ipynb")
nbdev_hq_path = Path(Path(".").resolve(), "example_nbs", "nbdev_high_quality.ipynb")
non_nbdev_path = Path(Path(".").resolve(), "example_nbs", "non_nbdev.ipynb")
non_nbdev_lq_path = Path(
    Path(".").resolve(), "example_nbs", "non_nbdev_low_quality.ipynb"
)
index_path = Path(Path(".").resolve(), "index.ipynb")

nbdev_nb = read_nb(nbdev_path)
nbdev_hq_nb = read_nb(nbdev_hq_path)
non_nbdev_nb = read_nb(non_nbdev_path)
non_nbdev_lq_nb = read_nb(non_nbdev_lq_path)
index = read_nb(index_path)

# `scilint_tidy` wrapper around no decisions version of nbqa

In [8]:
# | export


def run_nbqa_cmd(cmd):
    print(f"Running {cmd}")
    project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
    args = CLIArgs.parse_args([cmd, str(project_root)])
    configs = _get_configs(args, project_root)
    output_code = _main(args, configs)
    return output_code

In [9]:
project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
assert os.path.basename(project_root) == "scilint"

In [10]:
# | export


def tidy():
    tidy_tools = ["black", "isort", "autoflake"]
    [run_nbqa_cmd(c) for c in tidy_tools]

In [11]:
# | export


@call_parse
def scilint_tidy():
    tidy()

# Quality relevant data extraction

## Helpers

In [12]:
# | export


def get_func_defs(code, ignore_private_prefix=True):
    func_names = []
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.FunctionDef):
            inner_cond = (
                False if ignore_private_prefix and stmt.name.startswith("_") else True
            )
            if inner_cond:
                func_names.append(stmt.name)
    return func_names

In [13]:
test_code = """
x()
def y():
    pass
def z():
    def a():
        pass
class A():
    def b():
        pass
def blabla():
    return 1
def _hidden():
    return None
"""
func_defs = ["a", "b", "blabla", "y", "z"]
assert func_defs == sorted(get_func_defs(test_code))

In [14]:
# | export


def count_func_calls(code, func_defs):
    func_calls = Counter({k: 0 for k in func_defs})
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Call):
            func_name = stmt.func.id if "id" in stmt.func.__dict__ else stmt.func.attr
            if func_name in func_defs:
                if func_name in func_calls:
                    func_calls[func_name] += 1
    return func_calls

In [15]:
test_code = """self.hierarchical_topic_reduction(3); 
topic_reduction(3); 
lambda x: topic(x); 
hierarchical_topic_reduction[4]; 
hierarchical_topic_reduction(4); 
blabla()
"""
test_func_defs = [
    "topic",
    "topic_reduction",
    "blablabla",
    "hierarchical_topic_reduction",
]

In [16]:
assert count_func_calls(test_code, test_func_defs) == Counter(
    {
        "topic": 1,
        "topic_reduction": 1,
        "blablabla": 0,
        "hierarchical_topic_reduction": 2,
    }
)

In [17]:
nb_cell_code = r"""
def something():
    pass; pass # in x 2
    
%load_ext autoreload
%autoreload 2

!ls -l
if 1!= 2:
    print(4)
#| export

import pandas as pd # out
from sciflow.utils import lib_path, odbc_connect, query # out

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
nb_to_sagemaker_pipeline() # out
"""

In [18]:
# | export


def replace_ipython_magics(code):
    # Replace Ipython magic and shell command symbol with comment
    code = code.replace("%", "#")
    code = re.sub(r"^!", "#", code)
    return re.sub(r"\n\W?!", "\n#", code)

In [19]:
throws = False
try:
    assert ast.parse(nb_cell_code)
except SyntaxError:
    throws = True
assert throws
assert type(ast.parse(replace_ipython_magics(nb_cell_code))) == ast.Module

In [20]:
# | export


def safe_div(numer, denom):
    return 0 if denom == 0 else numer / denom

In [21]:
assert safe_div(1, 1) == 1
assert safe_div(2, 1) == 2
assert safe_div(1, 2) == 0.5
assert safe_div(0, 1) == 0
assert safe_div(1, 0) == 0
assert safe_div(10, 1) == 10

In [22]:
# | export


def get_cell_code(nb):
    pnb = nbformat.from_dict(nb)
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in pnb.cells
            if c["cell_type"] == "code"
        ]
    )
    return nb_cell_code

## 1. Calls-per-Function

In [23]:
# | export


def calls_per_func(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_func_defs(nb_cell_code)
    func_calls = count_func_calls(nb_cell_code, func_defs)
    return func_calls

In [24]:
# | export


def mean_cpf(nb):
    return pd.Series(calls_per_func(nb)).mean()

In [25]:
# | export


def median_cpf(nb):
    return pd.Series(calls_per_func(nb)).median()

In [26]:
assert mean_cpf(nbdev_nb).round(2) == 2.23
assert median_cpf(nbdev_nb) == 1

In [27]:
assert mean_cpf(read_nb(nbdev_path)).round(2) == 2.23
assert mean_cpf(read_nb(nbdev_hq_path)).round(2) == 2.5
assert mean_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert mean_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.62
assert pd.isnull(mean_cpf(index))

In [28]:
assert median_cpf(read_nb(nbdev_path)) == 1.0
assert median_cpf(read_nb(nbdev_hq_path)).round(2) == 1.5
assert median_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert median_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.0
assert pd.isnull(median_cpf(index))

## 2. Asserts-to-Function Ratio

In [29]:
asserted_code = r"""
def something():
    pass; pass # in x 2
    
assert True

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
assert len(x) > 2
assert something() is None # something +1

def tr():
    return True
    
def get_seg(num):
    return 2
    
assert(tr)
assert(tr()) # tr +1
assert(tr() == 4) # tr +1
assert(4 ==tr()) # tr +1
assert 0 != 0
assert "' '".join(tr(1)) == "00" # tr +1
assert len(get_seg(50)) == 50 # get_seg +1
assert max([int(x) for x in get_seg(100)]) == 99 # get_seg +1
"""

In [30]:
import nbformat as nbf

In [31]:
asserted_nb = nbf.v4.new_notebook()
asserted_nb["cells"] = [nbf.v4.new_code_cell(asserted_code)]

In [32]:
# | export


def afr(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":  # no code cells - metrics is not well defined
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    num_funcs = len(func_defs)

    assert_count = 0
    for stmt in ast.walk(ast.parse(nb_cell_code)):
        if isinstance(stmt, ast.Assert):
            assert_count += 1

    return safe_div(assert_count, num_funcs)

In [33]:
assert afr(nbdev_nb) > 1
assert afr(nbdev_hq_nb) > 1
assert afr(non_nbdev_nb) == 0
assert afr(non_nbdev_lq_nb) == 0
assert pd.isnull(afr(index))

## 3. In-line Asserts Per Function

In [34]:
# | export


def count_inline_asserts(code, func_defs):
    inline_func_asserts = Counter({k: 0 for k in func_defs})

    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Assert):
            for assert_st in ast.walk(stmt):
                if isinstance(assert_st, ast.Call):
                    func_name = (
                        assert_st.func.id
                        if "id" in assert_st.func.__dict__
                        else assert_st.func.attr
                    )
                    if func_name in inline_func_asserts:
                        inline_func_asserts[func_name] += 1
    return inline_func_asserts

In [35]:
# | export


def iaf(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    return count_inline_asserts(nb_cell_code, func_defs)

In [36]:
func_defs = get_func_defs(asserted_code)
inline_asserts_expected = Counter(
    {"something": 1, "tr": 4, "get_seg": 2, "nb_to_sagemaker_pipeline": 0}
)
inline_asserts_actual = count_inline_asserts(asserted_code, func_defs)

In [37]:
assert inline_asserts_actual == inline_asserts_expected

In [39]:
assert 0.0 == pd.Series(iaf(nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(nbdev_hq_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_lq_nb)).median()
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore', message='Mean of empty slice')
    assert pd.isnull(pd.Series(iaf(index)).median())

In [40]:
iaf(non_nbdev_nb)

Counter({'scalar': 0, 'py_advanced': 0, 'pandas': 0})

In [41]:
iaf(non_nbdev_lq_nb)

Counter({'get_traffic_text': 0,
         'get_experiment_segment': 0,
         'evaluate': 0,
         'serve_num_topics': 0,
         'get_num_topics': 0,
         'get_topic_sizes': 0,
         'get_topics': 0,
         'plot_wordcloud': 0})

In [42]:
assert inline_asserts_expected == iaf(asserted_nb)

In [43]:
# | export


def mean_iaf(nb):
    return pd.Series(iaf(nb)).mean()

In [64]:
# | export


def median_iaf(nb):
    with warnings.catch_warnings():
        warnings.filterwarnings(action='ignore', message='Mean of empty slice')
        return pd.Series(iaf(nb)).median()

## Full Code Coverage?

How does pytest-cov do it?

## 2. In-function Percentage

In [65]:
# | export


def calc_ifp(nb_cell_code):
    stmts_in_func = 0
    stmts_outside_func = 0
    for stmt in ast.walk(ast.parse(replace_ipython_magics(nb_cell_code))):
        if isinstance(stmt, ast.FunctionDef) and not stmt.name.startswith("_"):
            for body_item in stmt.body:
                stmts_in_func += 1
        elif isinstance(stmt, ast.Module):
            for body_item in stmt.body:
                if not isinstance(body_item, ast.FunctionDef):
                    stmts_outside_func += 1
    return (
        0
        if stmts_outside_func + stmts_in_func == 0
        else (stmts_in_func / (stmts_outside_func + stmts_in_func)) * 100
    )

In [66]:
assert (calc_ifp(nb_cell_code)) == (5 / (5 + 5)) * 100

In [67]:
# | export


def ifp(nb):
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in nb.cells
            if c["cell_type"] == "code"
        ]
    )
    if nb_cell_code == "":
        return np.nan
    return calc_ifp(nb_cell_code)

In [68]:
assert ifp(nbdev_nb) >= 0
assert ifp(nbdev_hq_nb) >= 0
assert ifp(non_nbdev_nb) >= 0
assert ifp(non_nbdev_lq_nb) >= 0
assert pd.isnull(ifp(index))

## 3. Markdown to Code Percent

In [69]:
# | export


def mcp(nb):
    md_cells = [c for c in nb.cells if c["cell_type"] == "markdown"]
    code_cells = [c for c in nb.cells if c["cell_type"] == "code"]
    num_code_cells = len(code_cells)
    if num_code_cells == 0:
        return np.nan
    num_md_cells = len(md_cells)
    return (
        100
        if num_code_cells == 0
        else (num_md_cells / (num_md_cells + num_code_cells)) * 100
    )

In [70]:
assert mcp(nbdev_nb) >= 0
assert mcp(nbdev_hq_nb) >= 0
assert mcp(non_nbdev_nb) >= 0
assert mcp(non_nbdev_lq_nb) >= 0
assert pd.isnull(mcp(index))

## 4. Total Code Length

In [71]:
# | export


def tcl(nb):
    return sum([len(c["source"]) for c in nb.cells if c["cell_type"] == "code"])

In [72]:
assert tcl(nbdev_nb) >= 50
assert tcl(nbdev_hq_nb) >= 50
assert tcl(non_nbdev_nb) >= 50
assert tcl(non_nbdev_lq_nb) >= 50

In [73]:
# | export


def lint_nb(
    nb_path,
    include_in_scoring,
    rounding_precision=3,
):
    nb = read_nb(nb_path)

    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        nb_cpf_median = round(median_cpf(nb), rounding_precision)
        nb_cpf_mean = round(mean_cpf(nb), rounding_precision)
        nb_ifp = round(ifp(nb), rounding_precision)
        nb_afr = round(afr(nb), rounding_precision)
        nb_iaf_median = round(median_iaf(nb), rounding_precision)
    nb_iaf_mean = round(mean_iaf(nb), rounding_precision)
    nb_mcp = round(mcp(nb), rounding_precision)
    nb_tcl = round(tcl(nb), rounding_precision)

    return (
        nb_cpf_median,
        nb_cpf_mean,
        nb_ifp,
        nb_afr,
        nb_iaf_median,
        nb_iaf_mean,
        nb_mcp,
        nb_tcl,
        include_in_scoring,
    )

In [74]:
# | export


# TODO generate and persist a new dataframe of warnings from this..


def format_quality_warning(metric, warning_data, warn_thresh, direction):
    for warning_row in warning_data.reset_index().itertuples():
        print(f'"{warning_row.index}" has: {metric} {direction} {warn_thresh}')

In [75]:
# | export


def get_excluded_paths(paths: Iterable[Path], exclude_pattern: str):
    excl_paths = []
    for ex_pattern in exclude_pattern.split(","):
        ex_path = Path(ex_pattern)
        if ex_path.exists():
            excl_paths.extend([p for p in paths if ex_pattern in str(p)])
        elif not ex_path.exists():
            raise ValueError(f"Path component: {ex_path} does not exist")
        else:
            raise ValueError(
                f"Invalid exclusion pattern: {ex_path} pattern is comma separrated list of 'dir/' for directories and 'name.ipynb' for specific notebook"
            )
    return excl_paths

In [76]:
paths = [Path(p) for p in nbglob()]
assert sorted(
    [
        p.name
        for p in get_excluded_paths(paths, exclude_pattern="example_nbs/,index.ipynb")
    ]
) == sorted(
    [
        "non_nbdev_low_quality.ipynb",
        "nbdev_high_quality.ipynb",
        "non_nbdev.ipynb",
        "nbdev.ipynb",
        "index.ipynb",
    ]
)
assert sorted(
    [
        p.name
        for p in get_excluded_paths(
            paths, exclude_pattern="example_nbs/nbdev.ipynb,index.ipynb"
        )
    ]
) == sorted(["nbdev.ipynb", "index.ipynb"])

In [77]:
# | export


def lint_nbs(
    cpf_med_warn_thresh=1,
    cpf_mean_warn_thresh=1,
    ifp_warn_thresh=20,
    afr_warn_thresh=1,
    iaf_med_warn_thresh=0,
    iaf_mean_warn_thresh=0.5,
    mcp_warn_thresh=5,
    tcl_warn_thresh=30000,
    rounding_precision=3,
    csv_out_path="/tmp/scilint.csv",
    exclusions=None,
):
    nb_paths = [Path(p) for p in nbglob()]

    excluded_paths = None
    if exclusions is not None:
        excluded_paths = get_excluded_paths(nb_paths, exclude_pattern=exclusions)

    lt_metric_cols = [
        "cpf_median",
        "cpf_mean",
        "in_function_pct",
        "asserts_function_ratio",
        "iaf_median",
        "iaf_mean",
        "markdown_code_pct",
    ]
    gt_metric_cols = ["total_code_len"]
    lt_metrics_thresholds = [
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
    ]
    gt_metrics_thresholds = [tcl_warn_thresh]
    results = []
    nb_names = []
    for nb_path in nb_paths:
        include_in_scoring = True
        if exclusions is not None:
            include_in_scoring = False if nb_path in excluded_paths else True

        nb_names.append(nb_path.stem)
        results.append(lint_nb(nb_path, include_in_scoring, rounding_precision))
    lint_report = pd.DataFrame.from_records(
        data=results,
        index=nb_names,
        columns=lt_metric_cols + gt_metric_cols + ["include_in_scoring"],
    ).sort_values(["cpf_median", "markdown_code_pct"], ascending=False)

    # Calculate warnings only from notebooks included in scoring
    scoring_report = lint_report[lint_report.include_in_scoring].copy()

    num_warnings = calculate_warnings(
        scoring_report,
        lt_metric_cols,
        lt_metrics_thresholds,
        gt_metric_cols,
        gt_metrics_thresholds,
    )

    lint_report.to_csv(csv_out_path)

    return lint_report, num_warnings

In [78]:
# | export


def calculate_warnings(
    scoring_report,
    lt_metric_cols,
    lt_metrics_thresholds,
    gt_metric_cols,
    gt_metrics_thresholds,
):
    # TODO tidy this up to usual config dict
    print("\n*********************Begin Scilint Report*********************")
    issues_raised = False
    num_warnings = 0

    for lt_metric_col, lt_metrics_threshold in zip(
        lt_metric_cols, lt_metrics_thresholds
    ):
        metrics_series = scoring_report[lt_metric_col]
        warning_data = metrics_series[metrics_series < lt_metrics_threshold]

        num_warnings += len(warning_data)
        if num_warnings > 0:
            issues_raised = True
        format_quality_warning(
            lt_metric_col,
            warning_data,
            lt_metrics_threshold,
            direction="<",
        )
    for gt_metric_col, gt_metrics_threshold in zip(
        gt_metric_cols, gt_metrics_thresholds
    ):
        metrics_series = scoring_report[gt_metric_col]
        warning_data = metrics_series[metrics_series > gt_metrics_threshold]
        num_warnings += len(warning_data)
        if num_warnings > 0:
            issues_raised = True
        format_quality_warning(
            gt_metric_col,
            warning_data,
            gt_metrics_threshold,
            direction=">",
        )
    if not issues_raised:
        print("No issues found")
    print("*********************End Scilint Report***********************")
    return num_warnings

In [79]:
lint_report, num_warnings = lint_nbs()
assert num_warnings == 5


*********************Begin Scilint Report*********************
"non_nbdev_low_quality" has: asserts_function_ratio < 1
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev_low_quality" has: iaf_mean < 0.5
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
*********************End Scilint Report***********************


In [80]:
lint_report

,cpf_median,cpf_mean,in_function_pct,asserts_function_ratio,iaf_median,iaf_mean,markdown_code_pct,total_code_len,include_in_scoring
scilint,2.0,3.250,47.115,2.036,0.0,1.750,17.284,24760,True
nbdev_high_quality,1.5,2.500,44.118,1.667,0.0,1.000,30.769,4978,True
nbdev,1.0,2.231,50.725,1.308,0.0,0.846,30.769,4918,True
non_nbdev_low_quality,1.0,1.625,45.000,0.000,0.0,0.000,15.789,2955,True
non_nbdev,1.0,1.000,35.714,0.000,0.0,0.000,0.000,1233,True
index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True


In [81]:
_, num_warnings = lint_nbs(exclusions="""example_nbs/,index.ipynb""")
assert num_warnings == 0


*********************Begin Scilint Report*********************
No issues found
*********************End Scilint Report***********************


In [82]:
_, num_warnings = lint_nbs(
    exclusions="""example_nbs/non_nbdev_low_quality.ipynb,index.ipynb"""
)
assert num_warnings == 3


*********************Begin Scilint Report*********************
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
*********************End Scilint Report***********************


In [83]:
_, num_warnings = lint_nbs(exclusions="""example_nbs/non_nbdev_low_quality.ipynb""")
assert num_warnings == 3


*********************Begin Scilint Report*********************
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
*********************End Scilint Report***********************


# Temporary nbdev library internal recreation

Due to an issue with argument chaining using the `fastcore` `@call_parse` decorator we are recreating some `nbdev` functionality here but without using that decorator. Come back to remove this when issue is resolved.

In [84]:
# | export


def _nbdev_lib_export():
    if os.environ.get("IN_TEST", 0):
        return
    files = nbglob(path=None, as_path=True).sorted("name")

    for f in files:
        nb_export(f, procs=None)
    add_init(get_config().lib_path)
    _build_modidx()

In [85]:
# | export


def _nbdev_lib_test(
    path=None,
    flags="",
    ignore_fname=".notest",
    n_workers: int = None,  # Number of workers
    timing: bool = False,  # Time each notebook to see which are slow
    do_print: bool = False,  # Print start and end of each notebook
    pause: float = 0.01,  # Pause time (in seconds) between notebooks to avoid race conditions
):
    "Test in parallel notebooks matching `path`, passing along `flags`"
    skip_flags = get_config().tst_flags.split()
    force_flags = flags.split()
    files = nbglob(path, as_path=True)
    from nbdev.test import _keep_file, test_nb

    files = [f.absolute() for f in sorted(files) if _keep_file(f, ignore_fname)]
    if len(files) == 0:
        return print("No files were eligible for testing")

    from fastcore.basics import IN_NOTEBOOK, num_cpus
    from fastcore.foundation import working_directory
    from fastcore.parallel import parallel

    if n_workers is None:
        n_workers = 0 if len(files) == 1 else min(num_cpus(), 8)
    if IN_NOTEBOOK:
        kw = {"method": "spawn"} if os.name == "nt" else {"method": "forkserver"}
    else:
        kw = {}
    wd_pth = get_config().nbs_path
    with working_directory(wd_pth if (wd_pth and wd_pth.exists()) else os.getcwd()):
        results = parallel(
            test_nb,
            files,
            skip_flags=skip_flags,
            force_flags=force_flags,
            n_workers=n_workers,
            basepath=get_config().config_path,
            pause=pause,
            do_print=do_print,
            **kw,
        )
    passed, times = zip(*results)
    if all(passed):
        print("Success.")
    else:
        _fence = "=" * 50
        failed = "\n\t".join(f.name for p, f in zip(passed, files) if not p)
        sys.stderr.write(
            f"\nnbdev Tests Failed On The Following Notebooks:\n{_fence}\n\t{failed}\n"
        )
        sys.exit(1)
    if timing:
        for i, t in sorted(enumerate(times), key=lambda o: o[1], reverse=True):
            print(f"{files[i].name}: {int(t)} secs")

In [86]:
# | export


def _nbdev_lib_clean():
    "Clean all notebooks in `fname` to avoid merge conflicts"
    # Git hooks will pass the notebooks in stdin
    from fastcore.basics import partial
    from fastcore.xtras import globtastic
    from nbdev.clean import _nbdev_clean
    from nbdev.clean import process_write

    _clean = partial(_nbdev_clean, clear_all=None)
    _write = partial(process_write, warn_msg="Failed to clean notebook", proc_nb=_clean)

    fname = get_config().nbs_path
    for f in globtastic(fname, file_glob="*.ipynb", skip_folder_re="^[_.]"):
        _write(f_in=f)

In [87]:
# | export


def _lint(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    lint_report, num_warnings = lint_nbs(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
    )
    if fail_over == -1:
        print("Linting outcome ignored as fail_over set to -1")
    elif num_warnings > fail_over:
        print(
            f"Linting failed: total warnings ({num_warnings}) exceeded threshold ({fail_over})"
        )
        sys.exit(num_warnings)
    else:
        print("Linting succeeded")

In [88]:
# | export


def _build(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    tidy()
    # TODO replace these with nbdev library versions when call_parse issue is resolved
    _nbdev_lib_export()
    _nbdev_lib_test()
    _lint(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )
    # TODO replace these with nbdev library versions when call_parse issue is resolved
    _nbdev_lib_clean()

# Console Scripts

* `scilint_lint`
* `scilint_build`
* `scilint_ci`

In [89]:
# | export

@call_parse
def scilint_lint(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    _lint(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )

In [90]:
# | export


@call_parse
def scilint_build(cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1):
    
    _build(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )

In [91]:
# | export


@call_parse
def scilint_ci(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: int = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    _build(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )
    if not shutil.which("quarto"):
        print(
            "Quarto is not installed. A working quarto install is required for the CI build"
        )
        sys.exit(-1)
    nbdev_readme()
    nbdev_docs()